# Pingüinos en el océano austral: vulnerabilidad espacial con Python 🐧🌊

## Contexto general

Los pingüinos de Magallanes (*Spheniscus magellanicus*) se reproducen en la costa del sur de Sudamérica y forrajean (se alimentan) ampliamente sobre la **Plataforma Patagónica**.  
Como consumidores tope, su uso del espacio se solapa con múltiples presiones humanas:

- **Pesquerías industriales y costeras**  
- **Cambios en la temperatura superficial del mar (SST)** vinculados al cambio climático  
- **Contaminación y tráfico marítimo**, entre otras

En estudios recientes analizamos los movimientos de estos pingüinos mediante la instrumentacion de geolocalizadores y cuantificamos su solapamiento con presiones como el esfuerzo pesquero [Dodino et al. 2021](https://doi.org/10.1371/journal.pone.0256339). Tambien estudiamos la prevalencia de mercurio (Hg) en la zona [Fioramonti et al. 2022](https://doi.org/10.1016/j.marpolbul.2022.113365), [Lois et al. 2022](https://www.sciencedirect.com/science/article/abs/pii/S0025326X22008190), [Dodino et al. 2022](https://doi.org/10.1016/j.marpolbul.2021.113184). 

En este tutorial vamos a utilizar reproducir **la lógica de ese análisis**, integrando:

- Datos de posicionamiento de pingüinos  
- Campos de **TSM** (anomalías del periodo, tendencias, y variabilidad)  
- Esfuerzo pesquero  
- Contaminación por Hg

para construir un índice simple de **vulnerabilidad espacial**.

---

## Vulnerabilidad: sensibilidad + exposición

En ecología y conservación de la biodiversidad, la **vulnerabilidad** puede entenderse de una manera muy simple como la combinación de:

- **Sensibilidad:** cuán susceptible es el organismo frente a las presiones  
- **Exposición:** cuánto se solapa espacialmente con esas presiones  

Para mayor profundizacion recomiendo la revision espectacular, aunque ahora un poco desactualizada de [Foden et al. 2019](https://doi.org/10.1002/wcc.551)

En este ejercicio nos enfocaremos principalmente en la **exposición**, integrando múltiples capas ambientales y antrópicas para cada punto de forrajeo.

---

## Objetivos

1. Cargar y explorar un conjunto de posiciones de forrajeo de pingüinos.  
2. Obtener para cada punto distintas **métricas ambientales y antrópicas** y construir índices simples de **exposición**:
   - Anomalía, tendencia y variabilidad de la TSM  
   - Intensidad de pesca  
   - Contaminación (Hg)
3. Integrar estas presiones en un **índice de vulnerabilidad espacial**.  
4. Visualizar la vulnerabilidad en el espacio y comparar patrones entre colonias.

Buscamos generar una primera aproximación a **mapas de riesgo** utilizando Python. La idea es que sirva como base para pensar cualquier problema de conservación del óceano.

## 📦 Datos utilizados en este ejercicio

En este notebook vamos a trabajar con un conjunto de **datos sintéticos** que generé para representar de una manera realista y simplificada los movimientos de los pingüinos de Magallanes durante la **temporada reproductiva y post-reproductiva (muda)** en la Plataforma Patagónica y las presiones ambientales que podrían afectar su éxito reproductivo.

Todos los datos fueron generados para permitir explorar conceptos de **vulnerabilidad espacial** y "que todo de bien" sin enroscarnos en el preprocesamiento pesado. Los procesamientos de datos y la información generada en las otras sesiones de la OHW pueden servir de insumos para este tipo de aproximaciones de generación de **mapas de riesgo**.

Los datos están preprocesados y presentados en **archivos CSV**. Tienen tambien un **netCDF** para probar una opción para transformarlo en csv. 

---

### 🐧 1. `penguins_gps.csv`  
**Contenido:**  
Puntos diarios de forrajeo/movimiento de pingüinos de tres colonias reales:  
- Punta Tombo  
- Puerto Deseado  
- Isla de los Estados  

**Columnas principales:**  
- `lat`, `lon`: posición geográfica  
- `colonia`: colonia de origen  
- `distancia_colonia_km`: distancia desde la colonia (costo espacial)

**Representa:**  
Movimientos en verano (cría y muda).

---

### 🌡️ 2. `sst_sintetica_patagonia.csv`  
**Contenido:** campo 2D sintético en grilla lat–lon con variables derivadas de TSM satelital:

- `sst_trend_decade`: tendencia climática (°C/década)  
- `sst_anom`: anomalía de la temporada de estudio con respecto a la media climatológica (°C)  
- `sst_sd`: variabilidad interanual (desviación estándar, °C)

**Uso en el ejercicio:** Construcción del componente exposición a cambio en la SST (discutamos al final del ejercicio que variable usar o como "integrarlas")

---

### 🎣 3. `pesca_sintetica_patagonia.csv`  
**Contenido:** Mapa sintético de **esfuerzo pesquero (horas/temporada de estudio)**.

- `esfuerzo_pesca_horas`: intensidad de pesca  

**Uso en el ejercicio:** Construcción del componente de exposición a pesca (discutamos al final del ejercicio que temporada de pesca usar)

---

### 🧪 4. `hg_sintetica_patagonia.csv`  
**Contenido:**  Campo sintético de **contaminación por Hg (ppm)** con un máximo regional cerca de 54°S y gradiente hacia el norte y sur.

- `hg`: concentración de mercurio en presas (ppm)

**Uso:** Construcción del componente de exposición a "contaminación".

---

### 🌐 5. `MODISA_L3m_SST4_Monthly_9km_20171101-20180331.71W_55S_53W_29S.nc`  
**Contenido:**  
Archivo netCDF de SST media para la temporada de estudio.

**Demo:** Mostrar brevemente cómo podemos pasar de un netCDF a un CSV listo para análisis (`xarray` → `pandas`).

## imports para todo el notebook

In [ ]:
import os                          # Operar con archivos y directorios
import numpy as np                 # Operaciones numéricas y manejo de arrays
import pandas as pd                # Manejo de tablas y CSVs
import xarray as xr                # Lectura y manejo de archivos netCDF
from scipy.spatial import cKDTree  # Join espacial más cercano (nearest neighbor)
import matplotlib.pyplot as plt    # Gráficos y mapas!
import cartopy.crs as ccrs         # Proyecciones cartográficas
import cartopy.feature as cfeature # Costas, océanos, continentes, etc

## Cargamos los datos

### Demo netcdf a csv

In [ ]:
# Exploramos el netCDF con SST media para la temporada de estudio
nc_file = "MODISA_L3m_SST4_Monthly_9km_20171101-20180331.71W_55S_53W_29S.nc"

ds = xr.open_dataset(nc_file)
ds # a ver el dataset que tenemos?


In [ ]:
# Tomamos el nombre de la variable
var_name = list(ds.data_vars)[0]  # devolveme la primera variable (es STT), asi no tenemos que escribirla y equivocarnos
print("Variable seleccionada:", var_name)




In [ ]:

# llamamos un array a partir del nombre de la variable
da = ds[var_name]

var_label = "SST media para el periodo de estudio" #ponemos un nombre de fantasia para los labels

# Plot "crudo" con imshow (sin preocuparse demasiado por proyección, para "ver")
plt.figure(figsize=(6, 5))
plt.imshow(da.values, origin="lower")
plt.imshow(da.values)    # luego de correrlo por primera vez, comenten esta linea para entender que hace el "origin=" en la linea anterior
plt.colorbar(label=var_label)
plt.title(var_label)
plt.tight_layout()
plt.show()

#### ❓ Repasemos un poco con este primer plot "crudo" desde netCDF

1. **¿Por qué la imagen parece estar “al revés” o rotada verticalmente?**  
   - Chequear el origen del array (`origin="lower"` / `origin="upper"`). ¿Las latitudes van de sur → norte o de norte → sur en este archivo?

---

2. **¿En qué unidades está la barra de colores?**  
   - Una pista: revisar si el netCDF incluye metadata en los atributos (`ds[var_name].attrs`).

---

3. **¿Qué representan los valores del eje X (xticks) y del eje Y (yticks)?**  
   - ¿Corresponden a valores reales de lon/lat? ¿Qué son?

---

4. **¿Por qué aparecen zonas blancas?**  
   - ¿Qué valor tienen?


#### Convertimos xarray a csv

In [ ]:
# Convertimos a DataFrame --> 
# lat, lon, valor
# lat, lon, valor
# lat, lon, valor
# lat, lon, valor ......

df_sst = da.to_dataframe(name=var_name).reset_index()

# Guardamos a CSV
csv_file = "sst_modisa_demo.csv"
df_sst.to_csv(csv_file, index=False)

print(f"CSV generado: {csv_file}")
df_sst.head()

#### 📝 ¿Por qué no procesamos todo directamente en CSV?

Ahora que ya convertimos el netCDF a un CSV simple, corré la celda siguiente y pensá:

- **¿Por qué no usamos CSV para el preprocesamiento de datos de variables oceanográficas?**  
  (pistas: tamaño, estructura, metadata, máscaras,…)

- **¿Por qué *sí* tiene sentido pasar a CSV en esta etapa del ejercicio?**

In [ ]:
# esta funcion la traigo siempre! La uso mucho. 
def file_size_mb(path):
    size_bytes = os.path.getsize(path)
    return size_bytes / (1024 * 1024)  # en megas!

archivos = [
    "sst_modisa_demo.csv",
    "MODISA_L3m_SST4_Monthly_9km_20171101-20180331.71W_55S_53W_29S.nc"
]

# No suelo recomendar usar trys, pero en este caso me resulta util
for f in archivos:
    try:
        print(f"{f}: {file_size_mb(f):.2f} MB")
    except FileNotFoundError:
        print(f"{f}: no encuentro el archivo!!")


#### Ploteamos desde csv ahora

In [ ]:
# Cargamos el CSV generado a partir del netCDF
df = pd.read_csv("sst_modisa_demo.csv")

# Reconstruimos la grilla 2D lat–lon
lats = np.sort(df["lat"].unique())
lons = np.sort(df["lon"].unique())

LAT, LON = np.meshgrid(lats, lons, indexing="ij")
Z = df[var_name].values.reshape(len(lats), len(lons))

proj = ccrs.PlateCarree()

fig = plt.figure(figsize=(8, 7))
ax = plt.axes(projection=proj)

# Usamos el dominio total del propio archivo
ax.set_extent([lons.min(), lons.max(), lats.min(), lats.max()], crs=proj)

# Armamos un mapa "lindo"
cf = ax.contourf(
    LON, LAT, Z,
    levels=20,
    cmap="viridis",
    transform=proj
)

# Redondeamos los rangos al entero más cercano
lon_min = np.floor(lons.min())
lon_max = np.ceil(lons.max())

lat_min = np.floor(lats.min())
lat_max = np.ceil(lats.max())

# Para nuestra region separamos en 5 a los ticks!
lon_step = 5 if (lon_max - lon_min) > 6 else 1
lat_step = 5 if (lat_max - lat_min) > 6 else 1

# rezamos que quede bien para no tener que hacerlo a mano =)
xticks = np.arange(lon_min, lon_max + lon_step, lon_step)
yticks = np.arange(lat_min, lat_max + lat_step, lat_step)

ax.set_xticks(xticks, crs=proj)
ax.set_yticks(yticks, crs=proj)

# esto es para evitar el "clutering" que suele haber en los plots
ax.xaxis.tick_bottom()
ax.yaxis.tick_left()

ax.set_xlabel("Longitud")
ax.set_ylabel("Latitud")

# Barra de color
cbar = plt.colorbar(cf, ax=ax, shrink=0.8)
cbar.set_label(var_label)

ax.set_title(f"MODIS SST temporada de estudio desde csv")

plt.show()


## Exploramos las variables que vamos a usar!

- Cargamos todos los CSV y algunas funciones para simplificar el mapeo (aunque nunca lo logro honestamente)
- Mapas facetados para SST (trend, anom, sd).
- Un mapa para pesca y otro para Hg.

In [ ]:
# Cargar CSVs sintéticos
peng = pd.read_csv("penguins_gps.csv")
sst  = pd.read_csv("sst_sintetica_patagonia.csv")
pesca = pd.read_csv("pesca_sintetica_patagonia.csv")
hg   = pd.read_csv("hg_sintetica_patagonia.csv")

# simplifico algunos nombres de vars
pesca = pesca.rename(columns={"esfuerzo_pesca_horas": "pesca"})
hg    = hg.rename(columns={"hg_ppm": "hg"})

# df to grid (lat, lon, var)
def df_to_grid(df, varname):
    lats = np.sort(df["lat"].unique())
    lons = np.sort(df["lon"].unique())
    LAT, LON = np.meshgrid(lats, lons, indexing="ij")
    Z = df[varname].values.reshape(len(lats), len(lons))
    return LAT, LON, Z, lats, lons

# Rounded ticks
def nice_ticks(lats, lons):
    lon_min = np.floor(lons.min())
    lon_max = np.ceil(lons.max())
    lat_min = np.floor(lats.min())
    lat_max = np.ceil(lats.max())

    lon_step = 2 if (lon_max - lon_min) > 6 else 1
    lat_step = 2 if (lat_max - lat_min) > 6 else 1

    xticks = np.arange(lon_min, lon_max + lon_step, lon_step)
    yticks = np.arange(lat_min, lat_max + lat_step, lat_step)
    return xticks, yticks

    import pandas as pd
import cartopy.crs as ccrs

# Cargar puntos GPS de pingüinos
peng = pd.read_csv("penguins_gps.csv")

# Paleta por colonia
colores_colonias = {
    "Punta Tombo": "dimgrey",
    "Puerto Deseado": "darkgrey",
    "Isla de los Estados": "lightgrey",
}

# Función para superponer los puntos GPS de pinguinos.
def plot_pinguinos(ax):
    for col, sub in peng.groupby("colonia"):
        ax.scatter(
            sub["lon"], sub["lat"],
            s=12,
            color=colores_colonias.get(col, "k"),
            alpha=0.7,
            transform=ccrs.PlateCarree(),
            label=col,
            zorder=10
        )
    
    # Mostrar leyenda solo una vez
    # ax.legend(title="Colonia", loc="lower left")



In [ ]:
vars_sst = ["sst_trend_decade", "sst_anom", "sst_sd"]
titulos_sst = [
    "Tendencia SST (°C/década)",
    "Anomalía SST (°C)",
    "Desviación estándar SST (°C)",
]

# Elegimos el colormap según variable
def elegir_cmap(var):
    if var == "sst_anom":
        return "coolwarm"
    elif var == "sst_trend_decade":
        return "viridis"
    elif var == "sst_sd":
        return "Purples"
    else:
        return "viridis"

# Construimos base de la grilla
LAT_sst, LON_sst, _, lats_sst, lons_sst = df_to_grid(sst, vars_sst[0])
xticks_sst, yticks_sst = nice_ticks(lats_sst, lons_sst)

proj = ccrs.PlateCarree()

fig, axes = plt.subplots(
    1, 3,
    figsize=(18, 7),
    subplot_kw={"projection": proj},
    constrained_layout=True
)

for ax, var, titulo in zip(axes, vars_sst, titulos_sst):
    _, _, Z, _, _ = df_to_grid(sst, var)

    ax.set_extent(
        [lons_sst.min(), lons_sst.max(),
         lats_sst.min(), lats_sst.max()],
        crs=proj
    )

    # ---- Colormap correcto ----
    cmap = elegir_cmap(var)

    # ---- Mapa ----
    cf = ax.contourf(
        LON_sst, LAT_sst, Z,
        levels=20,
        cmap=cmap,
        transform=proj
    )

    plot_pinguinos(ax)
    
    ax.add_feature(
        cfeature.LAND.with_scale("10m"),
        facecolor="white",
        zorder=5
    )

    # ---- Ticks redondos ----
    ax.set_xticks(np.round(xticks_sst), crs=proj)
    ax.set_yticks(np.round(yticks_sst), crs=proj)
    ax.xaxis.tick_bottom()
    ax.yaxis.tick_left()

    ax.set_xlabel("Longitud")
    if ax is axes[0]:
        ax.set_ylabel("Latitud")

    ax.set_title(titulo)

    # ---- Colorbar horizontal debajo ----
    cbar = fig.colorbar(cf, ax=ax, orientation="horizontal", pad=0.1, shrink=0.8)
    cbar.set_label(titulo)

plt.show()


In [ ]:
# Pasamos de df a grilla 2D
LAT_pes, LON_pes, Z_pes, lats_pes, lons_pes = df_to_grid(pesca, "pesca")
xticks_pes, yticks_pes = nice_ticks(lats_pes, lons_pes)

proj = ccrs.PlateCarree()

fig = plt.figure(figsize=(8, 7))
ax = plt.axes(projection=proj)

# Extensión
ax.set_extent(
    [lons_pes.min(), lons_pes.max(),
     lats_pes.min(), lats_pes.max()],
    crs=proj
)

# Campo de pesca
cf = ax.contourf(
    LON_pes, LAT_pes, Z_pes,
    levels=20,
    cmap="magma",
    transform=proj
)

plot_pinguinos(ax)

# Continente encima
ax.add_feature(
    cfeature.LAND.with_scale("10m"),
    facecolor="white",
    zorder=5
)

# Ticks redondos solo abajo/izquierda
ax.set_xticks(np.round(xticks_pes), crs=proj)
ax.set_yticks(np.round(yticks_pes), crs=proj)
ax.xaxis.tick_bottom()
ax.yaxis.tick_left()

ax.set_xlabel("Longitud")
ax.set_ylabel("Latitud")

# Colorbar horizontal debajo
cbar = fig.colorbar(cf, ax=ax, orientation="horizontal", pad=0.12, shrink=0.7)
cbar.set_label("Esfuerzo de pesca (horas/temporada de estudio)")

ax.set_title("Esfuerzo de pesca\nPlataforma Patagónica")

plt.show()


In [ ]:
# Pasamos de df a grilla 2D
LAT_hg, LON_hg, Z_hg, lats_hg, lons_hg = df_to_grid(hg, "hg")
xticks_hg, yticks_hg = nice_ticks(lats_hg, lons_hg)

proj = ccrs.PlateCarree()

fig = plt.figure(figsize=(8, 7))
ax = plt.axes(projection=proj)

# Extensión espacial
ax.set_extent(
    [lons_hg.min(), lons_hg.max(),
     lats_hg.min(), lats_hg.max()],
    crs=proj
)

# Campo de Hg
cf = ax.contourf(
    LON_hg, LAT_hg, Z_hg,
    levels=20,
    cmap="inferno",
    transform=proj
)

# Continente encima (sin borders ni coastline)
ax.add_feature(
    cfeature.LAND.with_scale("10m"),
    facecolor="white",
    zorder=5
)

# 👉 Superponemos pingüinos
plot_pinguinos(ax)

# Ticks redondos sólo abajo/izquierda
ax.set_xticks(np.round(xticks_hg), crs=proj)
ax.set_yticks(np.round(yticks_hg), crs=proj)
ax.xaxis.tick_bottom()
ax.yaxis.tick_left()

ax.set_xlabel("Longitud")
ax.set_ylabel("Latitud")

# Colorbar horizontal
cbar = fig.colorbar(cf, ax=ax, orientation="horizontal", pad=0.12, shrink=0.7)
cbar.set_label("Hg en presas (ppm)")

ax.set_title("Concentración de Hg")

plt.show()


## 🐧📊 Exposición: ¿qué enfrentan realmente los pingüinos?

Hasta ahora vimos los mapas ambientales completos (background), pero los pingüinos **no usan toda el área**:  
usan solo una fracción del espacio, y su **exposición real** depende de *dónde están*.

En esta sección vamos a comparar:

- **Distribución de fondo de cada variable** (*background*, que hay "disponible" en toda la grilla del dataset)
- **Distribución experimentada por cada colonia** (valores asociados a los puntos GPS)

Esto permite responder preguntas clave de vulnerabilidad: ¿El ambiente que experimentan es extremo comparado con el disponible a nivel regional?

Este paso es fundamental para construir el índice de vulnerabilidad!!


In [ ]:

# Construimos un KDTree con la grilla de Hg
hg_coords = np.vstack([hg["lat"].values, hg["lon"].values]).T
tree_hg = cKDTree(hg_coords)

# Coordenadas de pingüinos
peng_coords = np.vstack([peng["lat"].values, peng["lon"].values]).T

# Para cada pingüino, índice del pixel de Hg más cercano
dist, idx = tree_hg.query(peng_coords)

# Creamos un df con Hg en los puntos de pingüinos
peng_hg = peng.copy()
peng_hg["hg"] = hg["hg"].values[idx]

peng_hg.head()

In [ ]:
## Empezamos por Hg como ejemplo

# Background ambiental: toda la grilla de Hg
bg_hg = hg["hg"].dropna()

fig, axes = plt.subplots(
    1, 4,
    figsize=(18, 4),
    sharey=True,
    constrained_layout=True
)

# Parámetros comunes
x_min, x_max = 0, 6   # ppm Hg
y_min, y_max = 0, 120 # frecuencia
bins = np.linspace(x_min, x_max, 30)

# ---- 1) Background ----
axes[0].hist(bg_hg, bins=bins, color="gray", alpha=0.8)
axes[0].set_title("Background")
axes[0].set_xlabel("Hg (ppm)")
axes[0].set_ylabel("Frecuencia")
axes[0].set_xlim(x_min, x_max)
axes[0].set_ylim(y_min, y_max)

# ---- 2–4) Colonias ----
for ax, (col, sub) in zip(axes[1:], peng_hg.groupby("colonia")):
    vals = sub["hg"].dropna()
    if len(vals) == 0:
        ax.text(0.5, 0.5, "Sin datos", ha="center", va="center", transform=ax.transAxes)
    else:
        ax.hist(vals, bins=bins, color="gray", alpha=0.8)

    ax.set_title(col)
    ax.set_xlabel("Hg (ppm)")
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(y_min, y_max)

plt.suptitle("Exposición a Hg: background vs colonias", fontsize=14)
plt.show()


In [ ]:

# --- 1) Construimos un grid ambiental combinado (SST + pesca + Hg) ---

# Aseguramos nombres
pesca = pesca.rename(columns={"esfuerzo_pesca_horas": "pesca"}) if "esfuerzo_pesca_horas" in pesca.columns else pesca
hg    = hg.rename(columns={"hg_ppm": "hg"}) if "hg_ppm" in hg.columns else hg

grid = (
    sst[["lat", "lon", "sst_trend_decade", "sst_anom", "sst_sd"]]
    .merge(pesca[["lat", "lon", "pesca"]], on=["lat", "lon"])
    .merge(hg[["lat", "lon", "hg"]],        on=["lat", "lon"])
    .sort_values(["lat", "lon"])
    .reset_index(drop=True)
)

# Signal-to-noise térmico
grid["sst_sn"] = grid["sst_trend_decade"] / grid["sst_sd"].replace(0, np.nan)

# --- 2) Join espacial: valores ambientales en cada pingüino ---

grid_coords = np.vstack([grid["lat"].values, grid["lon"].values]).T
tree = cKDTree(grid_coords)

peng_coords = np.vstack([peng["lat"].values, peng["lon"].values]).T
dist, idx = tree.query(peng_coords)

peng_env = peng.copy()
peng_env["sst_trend_decade"] = grid["sst_trend_decade"].values[idx]
peng_env["sst_sd"]           = grid["sst_sd"].values[idx]
peng_env["sst_sn"]           = peng_env["sst_trend_decade"] / peng_env["sst_sd"].replace(0, np.nan)
peng_env["pesca"]            = grid["pesca"].values[idx]
peng_env["hg"]               = grid["hg"].values[idx]

# --- 3) Preparamos datos para los histogramas ---

rows = [
    ("sst_sn", "Signal-to-noise térmico (trend / SD)"),
    ("pesca",  "Esfuerzo de pesca (horas/año)"),
    ("hg",     "Hg en presas (ppm)"),
]

col_order = ["Background", "Isla de los Estados", "Puerto Deseado", "Punta Tombo"]

# Dataset "background"
bg_data = {
    "sst_sn": grid["sst_sn"],
    "pesca":  grid["pesca"],
    "hg":     grid["hg"],
}

# Datos por colonia
colonias_order = ["Isla de los Estados", "Puerto Deseado", "Punta Tombo"]

colonia_groups = {
    col: peng_env[peng_env["colonia"] == col]
    for col in colonias_order
}

In [ ]:
## =============================
##   EXPOSICIÓN A SST Signal-to-Noise
##   Background vs colonias
## =============================

var = "sst_sn"
label = "SST signal-to-noise (Trend / SD)"

# Background
series_bg = bg_data[var].replace([np.inf, -np.inf], np.nan).dropna()

# Datos de colonias
series_cols = {
    col: colonia_groups[col][var].replace([np.inf, -np.inf], np.nan).dropna()
    for col in colonias_order
}

# Rango común en X
all_series = [series_bg] + list(series_cols.values())
all_series = [s for s in all_series if len(s) > 0]

x_min = min(s.min() for s in all_series)
x_max = max(s.max() for s in all_series)
x_pad = 0.05 * (x_max - x_min) if x_max > x_min else 1
x_min -= x_pad
x_max += x_pad

bins = np.linspace(x_min, x_max, 30)

# Rango común de Y
max_count = 0
for s in all_series:
    counts, _ = np.histogram(s, bins=bins)
    max_count = max(max_count, counts.max())
y_max = max_count * 1.2 if max_count > 0 else 1

# Figure
fig, axes = plt.subplots(1, 4, figsize=(18, 3), sharey=True, constrained_layout=True)

# Background
axes[0].hist(series_bg, bins=bins, color="gray", alpha=0.8)
axes[0].set_title("Background")
axes[0].set_xlim(x_min, x_max)
axes[0].set_ylim(0, y_max)
axes[0].set_xlabel(label)
axes[0].set_ylabel("Frecuencia")

# Colonias
for ax, col in zip(axes[1:], colonias_order):
    vals = series_cols[col]
    if len(vals) == 0:
        ax.text(0.5, 0.5, "Sin datos", ha="center", va="center", transform=ax.transAxes)
    else:
        ax.hist(vals, bins=bins, color="gray", alpha=0.8)
    ax.set_title(col)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(0, y_max)
    ax.set_xlabel(label)

plt.suptitle("Exposición a SST (signal-to-noise)", fontsize=14)
plt.show()


In [ ]:
## =============================
##        EXPOSICIÓN A PESCA
##   Background vs colonias
## =============================

var = "pesca"
label = "Esfuerzo de pesca (horas/año)"

series_bg = bg_data[var].dropna()

series_cols = {
    col: colonia_groups[col][var].dropna()
    for col in colonias_order
}

# Rangos por variable
all_series = [series_bg] + list(series_cols.values())
all_series = [s for s in all_series if len(s) > 0]

x_min = min(s.min() for s in all_series)
x_max = max(s.max() for s in all_series)
x_pad = 0.05 * (x_max - x_min)
x_min -= x_pad
x_max += x_pad
bins = np.linspace(x_min, x_max, 30)

# Y-range común
max_count = 0
for s in all_series:
    counts, _ = np.histogram(s, bins=bins)
    max_count = max(max_count, counts.max())
y_max = max_count * 1.2 if max_count > 0 else 1

fig, axes = plt.subplots(1, 4, figsize=(18, 3), sharey=True, constrained_layout=True)

# Background
axes[0].hist(series_bg, bins=bins, color="gray", alpha=0.8)
axes[0].set_title("Background")
axes[0].set_xlim(x_min, x_max)
axes[0].set_ylim(0, y_max)
axes[0].set_xlabel(label)
axes[0].set_ylabel("Frecuencia")

# Colonias
for ax, col in zip(axes[1:], colonias_order):
    vals = series_cols[col]
    if len(vals) == 0:
        ax.text(0.5, 0.5, "Sin datos", ha="center", va="center", transform=ax.transAxes)
    else:
        ax.hist(vals, bins=bins, color="gray", alpha=0.8)
    ax.set_title(col)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(0, 100) ## OJO que achuramos la distribucion background asi para que se vea algo de la distribucion de cada colonia
    ax.set_xlabel(label)

plt.suptitle("Exposición al esfuerzo pesquero", fontsize=14)
plt.show()


In [ ]:
## =============================
##           EXPOSICIÓN A Hg
##    Background vs colonias
## =============================

## ya lo vimos, pero vamos de nuevo para ser ordenados! =)

var = "hg"
label = "Hg (ppm)"

series_bg = bg_data[var].dropna()
series_cols = {
    col: colonia_groups[col][var].dropna()
    for col in colonias_order
}

# Rangos independientes por variable
all_series = [series_bg] + list(series_cols.values())
all_series = [s for s in all_series if len(s) > 0]

x_min = min(s.min() for s in all_series)
x_max = max(s.max() for s in all_series)
x_pad = 0.05 * (x_max - x_min)
x_min -= x_pad
x_max += x_pad
bins = np.linspace(x_min, x_max, 30)

# Y-range común
max_count = 0
for s in all_series:
    counts, _ = np.histogram(s, bins=bins)
    max_count = max(max_count, counts.max())
y_max = max_count * 1.2 if max_count > 0 else 1

fig, axes = plt.subplots(1, 4, figsize=(18, 3), sharey=True, constrained_layout=True)

# Background
axes[0].hist(series_bg, bins=bins, color="gray", alpha=0.8)
axes[0].set_title("Background")
axes[0].set_xlim(x_min, x_max)
axes[0].set_ylim(0, y_max)
axes[0].set_xlabel(label)
axes[0].set_ylabel("Frecuencia")

# Colonias
for ax, col in zip(axes[1:], colonias_order):
    vals = series_cols[col]
    if len(vals) == 0:
        ax.text(0.5, 0.5, "Sin datos", ha="center", va="center", transform=ax.transAxes)
    else:
        ax.hist(vals, bins=bins, color="gray", alpha=0.8)
    ax.set_title(col)
    ax.set_xlim(x_min, x_max)
    ax.set_ylim(0, y_max)
    ax.set_xlabel(label)

plt.suptitle("Exposición a Hg", fontsize=14)
plt.show()


## Construimos un índice integrado de exposición

Ya exploramos cada presión por separado y comparamos el **background ambiental** de toda el área, la **exposición real** que experimenta cada colonia (basado en los valores en cada punto GPS).

El siguiente paso es construir un **índice integrado de exposición**, que combine estas presiones en una sola métrica por punto.

En una primera aproximación vamos a:

1. Normalizar cada presión al rango 0–1
2. Asumir que todas pesan por igual
3. Definir el índice como la suma de las exposiciones normalizadas (rango 0-3)

**Exposición integrada = E_*SST* + E_*pesca* + E_*Hg***

Pensemos (ahora o al final) que problemas pueden traer estas decisiones que tomamos.

In [ ]:
# Variables de exposición que vamos a integrar
expo_vars = ["sst_sn", "pesca", "hg"]

def normalizar(col):
    col = col.replace([np.inf, -np.inf], np.nan)
    vmin, vmax = col.min(), col.max()
    if pd.isna(vmin) or pd.isna(vmax) or vmax == vmin:
        return col * 0  # todo cero si no hay rango
    return (col - vmin) / (vmax - vmin)

# Creamos versiones normalizadas en peng_env
peng_expo = peng_env.copy()

for var in expo_vars:
    peng_expo[f"{var}_norm"] = normalizar(peng_expo[var])

peng_expo[[f"{v}_norm" for v in expo_vars]].describe()

In [ ]:
# Índice integrado de exposición (pesos iguales)
peng_expo["expo_index"] = (
    peng_expo["sst_sn_norm"] +
    peng_expo["pesca_norm"] +
    peng_expo["hg_norm"]
)

peng_expo.groupby("colonia")["expo_index"].describe()

In [ ]:
## Mapeamos, a ver???
 
proj = ccrs.PlateCarree()

fig = plt.figure(figsize=(8, 8))
ax = plt.axes(projection=proj)

# Usamos el rango espacial de los pingüinos
lon_min = peng_expo["lon"].min() - 1
lon_max = peng_expo["lon"].max() + 1
lat_min = peng_expo["lat"].min() - 1
lat_max = peng_expo["lat"].max() + 1

ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=proj)

# Continente
ax.add_feature(
    cfeature.LAND.with_scale("10m"),
    facecolor="lightgray",
    zorder=5
)

# Scatter de índice de exposición
sc = ax.scatter(
    peng_expo["lon"], peng_expo["lat"],
    c=peng_expo["expo_index"],
    s=20,
    cmap="Reds",
    transform=proj,
    zorder=10,
    edgecolor="k",
    linewidth=0.2,
)

# Ticks redondos
xticks = np.linspace(lon_min, lon_max, 5)
yticks = np.linspace(lat_min, lat_max, 5)
ax.set_xticks(xticks, crs=proj)
ax.set_yticks(yticks, crs=proj)
ax.xaxis.tick_bottom()
ax.yaxis.tick_left()

ax.set_xlabel("Longitud")
ax.set_ylabel("Latitud")
ax.set_title("Índice integrado de exposición\npara pingüinos de Magallanes")

cbar = plt.colorbar(sc, ax=ax, orientation="horizontal", pad=0.12, shrink=0.5)
cbar.set_label("Índice de exposición (0–3)")

plt.show()


In [ ]:
# Orden deseado
orden_colonias = ["Isla de los Estados", "Puerto Deseado", "Punta Tombo"]

# Extraer valores en ese orden
data = [peng_expo[peng_expo["colonia"] == col]["expo_index"].dropna().values
        for col in orden_colonias]

plt.figure(figsize=(8, 4))

# --- Boxplot ---
bp = plt.boxplot(
    data,
    labels=orden_colonias,
    patch_artist=True,
    medianprops=dict(color="black"),
    boxprops=dict(facecolor="lightgray", color="black"),
    whiskerprops=dict(color="black"),
    capprops=dict(color="black"),
)

# --- Jitter points ---
for i, vals in enumerate(data, start=1):
    x_jitter = i + np.random.uniform(-0.1, 0.1, size=len(vals))
    plt.scatter(
        x_jitter, vals,
        color="black",
        alpha=0.3,
        s=12
    )

plt.ylabel("Índice integrado de exposición")
plt.xlabel("Colonia")
plt.ylim(0, 3)
plt.title("Índice integrado de exposición por colonia")

plt.show()


## 🧩 Integrar presiones para entender la vulnerabilidad de las especies

En este ejercicio recorrimos un flujo de análisis espacial basado en datos y preguntas realistas sobre la conservación de una especie de pingüinos relativamente bien estudiada.

- exploramos datos ambientales (SST, pesca, Hg) y entendimos cómo varían en el espacio,
- analizamos cómo se superponen con las áreas de uso de los pingüinos,
- comparamos cada presión entre el *background* ambiental y el uso que hacen los pingüinos de cada colonia,
- y finalmente construimos un **índice integrado de exposición** a las presiones estudiadas.

Este índice busca una **primera aproximación cuantitativa** a cómo múltiples amenazas pueden sumarse sobre las poblaciones.

El mensaje central es que **ninguna presión actúa sola**: la relevancia ecológica está en su *combinación*, y en cómo cada colonia o población queda expuesta a configuraciones espaciales muy distintas.

Para profundizar este tipo de aproximaciones debemos pensar, por un lado, si deberíamos aplicar pesos distintos a cada presión (muy probablemente este sea el caso en la mayoría de las situaciones) y, a la vez, intentar sintetizar toda la información disponible al momento del análisis.

Por otro lado, a este esquema se le debe sumar la dimensión de **sensibilidad** —fisiología, demografía, alimentación, historia evolutiva— para avanzar hacia un marco más completo de **vulnerabilidad**. Pero incluso con esta simplificación, ya se puede visualizar cómo un enfoque espacial integrador ayuda a identificar zonas críticas, guiar monitoreos y apoyar decisiones de conservación basadas en evidencia.

Este es el corazón del análisis interdisciplinario: **unir datos ambientales, presiones humanas y movimiento y distribución de especies para entender quién está en riesgo, cuándo, dónde y por qué.**

